<a href="https://colab.research.google.com/github/sallytakeda/python-excel-kadai/blob/main/auto_order_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import defaultdict

# ファイル名一覧
order_files = [
    "order_A_20230524.xlsx",
    "order_B_20230524.xlsx",
    "order_C_20230524.xlsx",
    "order_D_20230524.xlsx"
]
# ======== 集計処理 ========
# 各注文ファイルを読み込み、1つのDataFrameにまとめる
df_list = []
for f in order_files:
    path =  f
    df = pd.read_excel(path)
    df_list.append(df)

# 全部の注文を縦に結合
all_orders = pd.concat(df_list, ignore_index=True)

print(all_orders)

# 野菜ごとに合計
summary = all_orders.sum()

print(summary)



    トマト  キャベツ  レタス   白菜  ほうれん草   大根  ニンジン
0  13.0   6.0   11  8.0    9.0  NaN   NaN
1   NaN   NaN   12  8.0   10.0  7.0  13.0
2   6.0   NaN   11  9.0    NaN  8.0  12.0
3  12.0  15.0    8  NaN    4.0  NaN   7.0
トマト      31.0
キャベツ     21.0
レタス      42.0
白菜       25.0
ほうれん草    23.0
大根       15.0
ニンジン     32.0
dtype: float64


In [2]:
import pandas as pd

# 在庫表ファイルの読み込み
inventory_file = "inventory.xlsx"
df = pd.read_excel(inventory_file)

# 最終行の抽出
latest_inventory = df.iloc[-1,2:]

print(latest_inventory)

トマト       91
キャベツ      73
レタス      103
白菜        84
ほうれん草     75
大根        48
ニンジン      50
Name: 13, dtype: object


In [3]:
# @title
remaining_stock=latest_inventory-summary
print(remaining_stock)

# pickup.xlsxからしきい値を読み込む
pickup_df = pd.read_excel("pickup.xlsx", header=None)

# 野菜名・しきい値・発注量の抽出
vegetables = pickup_df.loc[0]
thresholds = pickup_df.loc[1]
addtionals = pickup_df.loc[2]


# print(vegetables)
# print(thresholds)
# print(addtionals)

# 発注が必要な野菜を特定
reorder_targets = {}
for veg, threshold, addtional in zip(vegetables, thresholds, addtionals):
    stock = remaining_stock.get(veg, None)
    if stock is not None and stock < threshold:
        reorder_targets[veg] = {
            "在庫": stock,
            "しきい値": threshold,
            "追加数": addtional
        }

# 結果表示
print("在庫がしきい値を下回った野菜一覧:")
for veg, info in reorder_targets.items():
    print(f"{veg}: 在庫 {info['在庫']} < しきい値 {info['しきい値']} → 発注対象(発注数{info['追加数']})")



トマト      60.0
キャベツ     52.0
レタス      61.0
白菜       59.0
ほうれん草    52.0
大根       33.0
ニンジン     18.0
dtype: object
在庫がしきい値を下回った野菜一覧:
ニンジン: 在庫 18.0 < しきい値 40 → 発注対象(発注数80)


In [4]:
# メール文のテンプレート生成
def generate_order_email(reorder_dict):
    email_body = "件名：野菜の追加発注のご連絡\n\n"
    email_body += "いつもお世話になっております。\n"
    email_body += "以下の野菜について、在庫がしきい値を下回ったため、追加発注をお願い申し上げます。\n\n"
    email_body += "【発注内容】\n"

    for vegetable, amount in reorder_dict.items():
        email_body += f"・{vegetable}: {amount} \n"

    email_body += "\nお手数ですが、明日午前中までの納品をお願いできれば幸いです。\n"
    email_body += "何卒よろしくお願いいたします。\n\n"
    email_body += "野菜卸売り会社\n担当：佐織\n"

    return email_body

# 発注量を計算してメール用の辞書を作成
reorder_list = {}
for veg, info in reorder_targets.items():
    order_amount = info["追加数"]
    reorder_list[veg] = order_amount

# メール文の表示
order_email = generate_order_email(reorder_list)
print(order_email)

件名：野菜の追加発注のご連絡

いつもお世話になっております。
以下の野菜について、在庫がしきい値を下回ったため、追加発注をお願い申し上げます。

【発注内容】
・ニンジン: 80 

お手数ですが、明日午前中までの納品をお願いできれば幸いです。
何卒よろしくお願いいたします。

野菜卸売り会社
担当：佐織



In [5]:
import pandas as pd
from datetime import datetime

# === ① 既存の在庫表を読み込み ===
inventory_file = "inventory.xlsx"
inventory_df = pd.read_excel(inventory_file)

# === ② 最新の在庫情報と注文合計を取得 ===
# 最新の在庫（前回の在庫）
latest_inventory = inventory_df.iloc[-1, 2:]  # 2列目以降が野菜列

# 注文合計（前工程で作成済みの summary を使用）
# summary = all_orders.sum() が前提

# === ③ 最新在庫を計算 ===
updated_inventory = latest_inventory - summary

# === ④ 新しい行を作成 ===
today = datetime.today().strftime("%Y-%m-%d")
new_row = pd.DataFrame([[len(inventory_df)+1, today] + updated_inventory.tolist()],
                       columns=inventory_df.columns)

# === ⑤ 在庫表に追加して保存 ===
inventory_df = pd.concat([inventory_df, new_row], ignore_index=True)
inventory_df.to_excel(inventory_file, index=False)

print("✅ 在庫表を更新しました。")

✅ 在庫表を更新しました。


In [6]:
# auto_order.py
import pandas as pd
from datetime import datetime

# === 1. 注文ファイルの集計 ===
order_files = ["order_A_20230524.xlsx", "order_B_20230524.xlsx", "order_C_20230524.xlsx", "order_D_20230524.xlsx"]
df_list = [pd.read_excel(f) for f in order_files]
all_orders = pd.concat(df_list, ignore_index=True)
summary = all_orders.sum()

# === 2. 在庫更新 ===
inventory_file = "inventory.xlsx"
inventory_df = pd.read_excel(inventory_file)
latest_inventory = inventory_df.iloc[-1, 2:]
updated_inventory = latest_inventory - summary
today = datetime.today().strftime("%Y-%m-%d")
new_row = pd.DataFrame([[len(inventory_df)+1, today] + updated_inventory.tolist()], columns=inventory_df.columns)
inventory_df = pd.concat([inventory_df, new_row], ignore_index=True)
inventory_df.to_excel(inventory_file, index=False)

# === 3. 発注判定とメール文生成 ===
pickup_df = pd.read_excel("pickup.xlsx", header=None)
vegetables = pickup_df.loc[0]
thresholds = pickup_df.loc[1]
remaining_stock = updated_inventory
reorder_targets = {}
for veg, threshold in zip(vegetables, thresholds):
    stock = remaining_stock.get(veg, None)
    if stock is not None and stock < threshold:
        reorder_targets[veg] = {"在庫": stock, "しきい値": threshold}

reorder_list = {veg: info["しきい値"] - info["在庫"] for veg, info in reorder_targets.items()}

def generate_order_email(reorder_dict):
    email_body = "件名：野菜の追加発注のご連絡\n\n"
    email_body += "いつもお世話になっております。\n"
    email_body += "以下の野菜について、在庫がしきい値を下回ったため、追加発注をお願い申し上げます。\n\n"
    email_body += "【発注内容】\n"
    for vegetable, amount in reorder_dict.items():
        email_body += f"・{vegetable}: {amount} kg\n"
    email_body += "\nお手数ですが、明日午前中までの納品をお願いできれば幸いです。\n"
    email_body += "何卒よろしくお願いいたします。\n\n"
    email_body += "野菜卸売り会社\n担当：佐織\n"
    return email_body

order_email = generate_order_email(reorder_list)
with open("order_email.txt", "w", encoding="utf-8") as f:
    f.write(order_email)